# OLlama
What it is Ollama
Ollama is a streamlined tool for running open-source LLMs locally, including Mistral and Llama 2. Ollama bundles model weights, configurations, and datasets into a unified package managed by a Modelfile.

https://klu.ai/glossary/ollama

- https://ollama.com/
- https://github.com/ollama/ollama/tree/main
- https://github.com/valiantlynx/ollama-docker
- https://github.com/ollama/ollama/blob/main/docs/api.md
- https://python.langchain.com/docs/integrations/llms/ollama

Import from PyTorch , Safetensors or GGUF

GGUF (GPT-Generated Unified Format), introduced as a successor to GGML (GPT-Generated Model Language), was released on the 21st of August, 2023. This format represents a significant step forward in the field of language model file formats, facilitating enhanced storage and processing of large language models like GPT.
  
# Pinecone
https://www.pinecone.io/


In [93]:
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import warnings
warnings.filterwarnings("ignore")
import time
from pinecone import Pinecone
import tqdm
from dotenv import load_dotenv
import os

In [2]:

import requests

response = requests.get("https://www.gutenberg.org/cache/epub/64317/pg64317.txt")

In [3]:
book_complete_text = response.text
book_complete_text = book_complete_text[5:]
len(book_complete_text)

296884

In [4]:
file_path = "./book.txt"

with open(file_path, "w", encoding="utf-8") as f:
    f.write(book_complete_text)

In [5]:
with open(file_path, "r",  encoding="utf-8") as f:
    text = f.read()

text = text.replace('\t', ' ')

In [33]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", "\t"], chunk_size=1200, chunk_overlap=100)

In [34]:
documents = text_splitter.create_documents([text])
num_documents = len(documents)

print (f"Now our book is split up into {num_documents} documents")

Now our book is split up into 270 documents


In [35]:

len(documents)

270

In [38]:
embeddings = OllamaEmbeddings(model="llama2:7b-chat", show_progress=True, num_gpu=1)

In [39]:
import datetime
texts = [doc.page_content for doc in documents]

In [40]:
texts[0]

'Project Gutenberg eBook of The Great Gatsby\n\n    \n\nThis ebook is for the use of anyone anywhere in the United States and\n\nmost other parts of the world at no cost and with almost no restrictions\n\nwhatsoever. You may copy it, give it away or re-use it under the terms\n\nof the Project Gutenberg License included with this ebook or online\n\nat www.gutenberg.org. If you are not located in the United States,\n\nyou will have to check the laws of the country where you are located\n\nbefore using this eBook.\n\n\n\nTitle: The Great Gatsby\n\n\n\n\n\nAuthor: F. Scott Fitzgerald\n\n\n\nRelease date: January 17, 2021 [eBook #64317]\n\n                Most recently updated: February 2, 2024\n\n\n\nLanguage: English\n\n\n\nCredits: Produced by Alex Cabal for the Standard Ebooks project, based on a transcription produced for Project Gutenberg Australia.\n\n\n\n\n\n*** START OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***\n\n\n\n\n\n\n\n\n\n                           The Great Gatsby\n

In [41]:
vectors = []
doc_nt = []
for doc,i in zip(texts, range(len(texts))):
    emb = embeddings.embed_documents(texts= [doc])
    if emb[0] != None:
        vectors.append(emb[0])
    else:
        print(f"document {i}, not treated first 20 chars {doc[:20]}")
        doc_nt.append(i)

OllamaEmbeddings: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.57s/it]


In [45]:
doc_nt

[]

In [116]:
len(vectors[-1])

4096

In [117]:
len(vectors)

270

In [94]:
env_path = os.path.join( ".env")
load_dotenv(env_path)

True

In [99]:
PINECONE_API_KEY= os.getenv("PINECONE_API_KEY")

In [100]:
pinecone = Pinecone(api_key=PINECONE_API_KEY)

In [101]:
INDEX_NAME = "rag-demo"

In [102]:
index = pinecone.Index(INDEX_NAME)

In [118]:
prepped = []

for i , em, text in zip(range(len(vectors)), vectors, texts):
    v = { "id" : f"vec{i}",
        "values" : em,
        "metadata" : {"text": text}}
    prepped.append(v)
    if len(prepped) >= 25:
        index.upsert(prepped)
        prepped = []
index.upsert(prepped)    

{'upserted_count': 20}

In [104]:
def get_embeddings(articles, embeddings):
   return embeddings.embed_documents(texts= articles)

In [120]:
query = "who is the Great Gastby?"

embed = get_embeddings([query], embeddings)


OllamaEmbeddings: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.80s/it]


In [121]:
embed

[[2.1973037719726562,
  -0.028831105679273605,
  1.257165551185608,
  0.2976294457912445,
  -2.274399757385254,
  1.2986702919006348,
  1.1521027088165283,
  -0.6425160765647888,
  -0.6691870093345642,
  -0.33284738659858704,
  1.0663673877716064,
  -1.021161675453186,
  -2.0653533935546875,
  2.3137314319610596,
  -0.08360535651445389,
  -0.9427536725997925,
  -1.1726484298706055,
  -7.16681606718339e-05,
  0.7132120728492737,
  -2.7329018115997314,
  0.7454558610916138,
  -1.8717437982559204,
  0.8461197018623352,
  -1.9609076976776123,
  0.35728132724761963,
  -1.5483232736587524,
  1.0169352293014526,
  -1.1033215522766113,
  0.6614121198654175,
  -1.1513675451278687,
  1.5639063119888306,
  -1.5878559350967407,
  -0.5884114503860474,
  5.040070533752441,
  2.4061901569366455,
  -5.095932960510254,
  -0.714613139629364,
  0.8797063231468201,
  0.03512461483478546,
  -1.1228277683258057,
  -2.4081125259399414,
  -4.306117057800293,
  0.9974054098129272,
  -2.045844554901123,
  -0.46

In [122]:
res = index.query(vector=embed, top_k=3, include_metadata=True)


In [123]:
type(res)

pinecone.core.client.model.query_response.QueryResponse

In [124]:
response = res.to_dict()

In [125]:
response.keys()

dict_keys(['matches', 'namespace', 'usage'])

In [126]:
response['usage']

{'read_units': 6}

In [127]:
response['matches'][0]

{'id': 'vec237',
 'score': 0.587523818,
 'values': [],
 'metadata': {'text': 'He was so hard up he had to keep on wearing his uniform because he\n\ncouldn’t buy some regular clothes. First time I saw him was when he\n\ncame into Winebrenner’s poolroom at Forty-third Street and asked for a\n\njob. He hadn’t eat anything for a couple of days. ‘Come on have some\n\nlunch with me,’ I said. He ate more than four dollars’ worth of food\n\nin half an hour.”\n\n\n\n“Did you start him in business?” I inquired.\n\n\n\n“Start him! I made him.”\n\n\n\n“Oh.”\n\n\n\n“I raised him up out of nothing, right out of the gutter. I saw right\n\naway he was a fine-appearing, gentlemanly young man, and when he told\n\nme he was at Oggsford I knew I could use him good. I got him to join\n\nthe American Legion and he used to stand high there. Right off he did\n\nsome work for a client of mine up to Albany. We were so thick like\n\nthat in everything”—he held up two bulbous fingers—“always together.”\n\n\n\nI w

In [113]:
for r in response['matches']:
    print(f" Score {r.get('score')}, text {r.get('metadata').get('text')}")

 Score 0.587523818, text He was so hard up he had to keep on wearing his uniform because he

couldn’t buy some regular clothes. First time I saw him was when he

came into Winebrenner’s poolroom at Forty-third Street and asked for a

job. He hadn’t eat anything for a couple of days. ‘Come on have some

lunch with me,’ I said. He ate more than four dollars’ worth of food

in half an hour.”



“Did you start him in business?” I inquired.



“Start him! I made him.”



“Oh.”



“I raised him up out of nothing, right out of the gutter. I saw right

away he was a fine-appearing, gentlemanly young man, and when he told

me he was at Oggsford I knew I could use him good. I got him to join

the American Legion and he used to stand high there. Right off he did

some work for a client of mine up to Albany. We were so thick like

that in everything”—he held up two bulbous fingers—“always together.”



I wondered if this partnership had included the World’s Series

transaction in 1919.



“Now he’